In [ ]:
# Define the GloveRun class to load and interpret data

import os
import subprocess
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from math import exp, inf, log, sqrt
from functools import lru_cache

# Load embeddings: these are trained with the full "Wikipedia" configuration
cooccur1_path = "../GloVe/wikipedia/paper/cooccurrence.bin"
cooccur2_path = "../GloVe/wikipedia/paper/cooccurrence2.bin"
cooccur1_filt_path = "../GloVe/wikipedia/paper/cooccurrence.filt.bin"
cooccur2_filt_path = "../GloVe/wikipedia/paper/cooccurrence2.filt.bin"
vocab1_path = "../GloVe/wikipedia/paper/vocab.txt"
vocab2_path = "../GloVe/wikipedia/paper/vocab2.txt"
vector1_path = "../GloVe/wikipedia/paper/vectors.bin"
vector2_path = "../GloVe/wikipedia/paper/vectors2.bin"
vector1_size = 100
vector2_size = 100


class GloveRun:
  B: np.ndarray      # bias vector
  C: csr_matrix      # cooccurrence matrix
  Csum: np.ndarray   # sum of cooccurrence rows
  dictionary: list   # dictionary list (of strings)
  freq: list         # frequency list
  word: np.ndarray   # matrix of word vectors
  ctx: np.ndarray    # matrix of context vectors
  evec: np.ndarray   # matrix of "e" vectors (word+context)
  enorm: np.ndarray  # vector of norms of each "e" vector
  pairs: list        # list of word pairs
  df: pd.DataFrame   # pandas dataframe with results

  def __init__(self, name, cooccur_path, cooccur_filt_path, vocab_path, vector_path, vector_size):
    self.name = name
    self.cooccur_path = cooccur_path
    self.cooccur_filt_path = cooccur_filt_path
    self.vocab_path = vocab_path
    self.vector_path = vector_path
    self.vector_size = vector_size  # AKA "embedding dimension"

    self.B = None
    self.C = None
    self.Csum = None
    self.dictionary = None
    self.freq = None
    self.word = None
    self.ctx = None
    self.evec = None
    self.enorm = None
    self.pairs = None

  # Loading functions:

  def load_vocab(self):
    self.dictionary = []
    self.freq = []
    with open(self.vocab_path, "r") as f:
      for line in f:
        word, freq = line.split(' ')
        self.dictionary.append(word)
        self.freq.append(freq)
    self.vocab_size = len(self.dictionary)
    self.word_idx = {w: i for i, w in enumerate(self.dictionary)}

  def load_vectors(self):
    dt = np.dtype('<f8')
    arr = np.fromfile(self.vector_path, dtype=dt)
    vecs = arr.reshape((2*self.vocab_size, self.vector_size+1))
    word_mat, ctx_mat = np.split(vecs, 2)
    self.word = word_mat[:, :self.vector_size]
    self.ctx = ctx_mat[:, :self.vector_size]
    self.B = word_mat[:, self.vector_size]
    self.evec = self.word + self.ctx
    self.enorm = np.linalg.norm(self.evec, axis=1)
    # note: bias_ctx (aka b'), stored in ctx_mat[:, vector_size], is unused
    # (see the footnote on p. 5)


  def load_cooccurrences(self):
    if not os.path.isfile(self.cooccur_filt_path):
      # run filt-cooccur.sh to filter the cooccurrence matrix to only
      # the indices we are interested in: the indices of our word pairs
      indices = {i for p in self.pairs for i in p}

      with open(self.cooccur_filt_path + ".tmp", "wb") as f:
        print("calling filter subproces...")
        args = ["./filt-cooccur.sh", self.cooccur_path] + \
            [str(i) for i in indices]
        subprocess.run(args, stdout=f, check=True)
        print("subprocess exited normally.")

      os.rename(self.cooccur_filt_path + ".tmp", self.cooccur_filt_path)

    # read the filtered cooccurrence matrix
    # entries are stored in COO format (see CREC from GloVe/src/common.h)
    # we convert to CSR format for faster access
    dt = np.dtype([('i', '<i4'), ('j', '<i4'), ('x', '<f8')])
    arr = np.fromfile(self.cooccur_filt_path, dtype=dt)
    self.C = csr_matrix((arr['x'], (arr['i']-1, arr['j']-1)))
    self.Csum = self.C.sum(axis=1).A1

  def setup_pairs(self, str_pairs):
    self.pairs = [(self.word_idx[s], self.word_idx[t]) for s, t in str_pairs]

  # Helper functions:

  def model_f(self, u: int, v: int, c: float, epsilon: float) -> float:
    """ Optimized f() """
    logc = log(c) if c > 0 else -inf
    return max(logc-self.B[u]-self.B[v], epsilon)

  def M_row(self, u):
    """ Calculate a sparse row of M """
    C_row = self.C.getrow(u)
    cols = C_row.nonzero()[1]
    vals = np.fromiter((self.model_f(u, v, self.C[u, v], 0)
                        for v in cols), dtype=np.float64)
    return csr_matrix((vals, C_row.indices, C_row.indptr), C_row.shape)
  
  def get_coss(self, t):
    """ Compute the cosine similarity for each word relative to the target """
    dots = self.evec @ self.evec[t].transpose()
    return dots / (self.enorm[t]*self.enorm)

  # Similarity functions:

  def cos_sim(self, s, t):
    """ Calculate the embedding similarity between the two given words """
    es = self.evec[s]
    et = self.evec[t]
    return es.dot(et)/(np.linalg.norm(es)*np.linalg.norm(et))

  def sim1(self, s, t):
    """ Calculate the distributional sim1 """
    num = self.model_f(s, t, self.C[s, t], 0)
    den1 = self.model_f(s, t, self.Csum[s], exp(-60))
    den2 = self.model_f(s, t, self.Csum[t], exp(-60))
    return num/sqrt(den1*den2)

  @lru_cache(None) # worst-case runtime is O(vocab_size), worth memoizing
  def sim2(self, s, t):
    """ Calculate the distributional sim2 """
    Ms = self.M_row(s)
    Mt = self.M_row(t)
    dot = (Ms @ Mt.transpose())[0, 0]
    Ms_norm2 = Ms.power(2).sum()
    Mt_norm2 = Mt.power(2).sum()
    return dot/sqrt(Ms_norm2*Mt_norm2)

  def sim12(self, s, t):
    """ Calculate the distributional sim1+2 """
    return (self.sim1(s, t)+self.sim2(s, t))/2

  def rank(self, s, t):
    """ Calculate the rank of the given source word relative to the target word """
    coss = self.get_coss(t)
    # find number of words with a greater cos similarity than `s` to the target
    # (including `t` itself, so rank 1 is the closest word to `t` except `t` itself)
    return np.sum(coss > coss[s])

  # Rank lists:

  def top_ranked(self, t, n):
    """ Find the top n highest-ranked words relative to the given target """
    coss = self.get_coss(t)
    # find the indices of the n+1 words w/ the highest cos similarity
    res = np.argpartition(coss, -(n+1))[-(n+1):]
    # sort the indices by cosine similarity
    res = res[np.argsort(coss[res])]
    # return indices 0 through n-2 in descending order (index n-1 is `t`)
    return res[-2::-1]

  def top_ranked_str(self, t, n):
    """ Get the top-ranked strings for the given target """
    return [self.dictionary[i] for i in self.top_ranked(t, n)]


# GloVe-no attack
run1 = GloveRun("orig ", cooccur1_path, cooccur1_filt_path,
                vocab1_path, vector1_path, vector1_size)
# GloVe-paper
run2 = GloveRun("paper", cooccur2_path, cooccur2_filt_path,
                vocab2_path, vector2_path, vector2_size)
runs = [run1, run2]


In [ ]:
# Load all the input data

print("Loading vocab...")
for run in runs:
  run.load_vocab()

print("Load vectors...")
for run in runs:
  run.load_vectors()

print("Loading sequences...")
with open("sequences.txt", "r") as f:
  str_pairs = []
  # look for first-order sequences, ie. lines of the form "foo bar"
  for line in f:
    line = line.strip()
    if not line:
      continue
    space0 = line.index(" ")
    space1 = line.find(" ", space0+1)
    if space1 == -1 and (not str_pairs or line != str_pairs[-1]):
      str_pairs.append(line)
str_pairs = [s.split(" ") for s in str_pairs]
print(len(str_pairs))
print(str_pairs)

# Note: since GloVe-paper limits the vocabulary to the 400k most common words,
# it's possible for the attacker to select a pair that was not part of the
# victim's vocabulary. In our tests this only happened once. Either way, we
# remove any pairs that aren't in vocabulary of all of the runs.
i = 0
while i < len(str_pairs):
  s, t = str_pairs[i]
  for run in runs:
    if s not in run.dictionary or t not in run.dictionary:
      print("WARNING: Couldn't find pair", s, t)
      if i < len(str_pairs)-1:
        str_pairs[i] = str_pairs.pop()
      else:
        str_pairs.pop()
      i -= 1
      break
  i += 1

print(len(str_pairs))
print(str_pairs)

for run in runs:
  run.setup_pairs(str_pairs)
  print(run.name, run.pairs)

for run in runs:
  print(f"Loading cooccurrences for {run.name}...")
  run.load_cooccurrences()

print("Done!")

In [ ]:
# Calculate the results and store in dataframes for each run

from collections import defaultdict

measures = [("cos", GloveRun.cos_sim), ("sim1", GloveRun.sim1),
            ("sim2", GloveRun.sim2), ("sim1+2", GloveRun.sim12),
            ("rank", GloveRun.rank)]
data = [defaultdict(list) for _ in range(len(runs))]

for i, (str_s, str_t) in enumerate(str_pairs):
  print("Source:", str_s, "Target:", str_t)
  for j, run in enumerate(runs):
    if j > 0:
      print(f"{run.name}:")
    for desc, func in measures:
      val = func(run, *run.pairs[i])
      data[j][desc].append(val)
      if j > 0:
        val0 = data[0][desc][i]
        print(f"  {desc}: {val0} -> {val}, DIFF: {val-val0}")
  print()

for d, run in zip(data, runs):
  run.df = pd.DataFrame(d)


In [ ]:
# Print the top 10 highest-ranked words for each target, before and after retraining
for i, (str_s, str_t) in enumerate(str_pairs):
  print("Source:", str_s, "Target:", str_t)
  for run in runs:
    t = run.pairs[i][1]
    print(run.name + ":")
    for w in run.top_ranked_str(t, 10):
      print("  " + w)
    print()
  print()

In [ ]:
# Plot the increase in distributional proximity vs. increase in embedding
# proximity, similar to Figure A.2(c) in the original paper

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
x = run2.df["sim1+2"] - run1.df["sim1+2"]
y = run2.df["cos"] - run1.df["cos"]
plt.scatter(x, y)
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
print(np.corrcoef(x, y)[1, 0])

In [ ]:
# Calculate results similarly to Table V

pd.DataFrame({
    run.name: {
        "median rank": run.df["rank"].median(),
        "avg. increase in proximity": (run.df["sim1+2"] - runs[0].df["sim1+2"]).mean() if i != 0 else 0,
        "rank < 10": len(run.df[run.df["rank"] < 10])
    }
    for i, run in enumerate(runs)
}).transpose()


In [ ]:
plt.xlabel("increase in distributional proximity (sim1+2)")
plt.hist(run2.df["sim1+2"]-run1.df["sim1+2"])

In [ ]:
plt.xlabel("increase in embedding proximity (cosine similarity)")
plt.hist(run2.df["cos"]-run1.df["cos"])